# Задача
Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE

В данном ноутбуке мы выбирали лучшую модель для предсказания.


# Загрузка данных

In [1]:
#Импорт библиотек
import numpy as np 
import pandas as pd 
from pandas import Series

import re
import datetime

from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import train_test_split, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

from tqdm.notebook import tqdm
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor

In [2]:
#Импорт данных из соревнования
data_sample = pd.read_csv('sample_submission.csv')

#Импорт обработанных данных
data_full = pd.read_csv('data_full_EDA.csv') 

#Выбранные признаки в EDA
data_columns = pd.read_csv('data_full_columns.csv') 

cat_cols = data_columns[data_columns.column_type == 'cat'].column_name.values
bin_cols = data_columns[data_columns.column_type == 'bin'].column_name.values
num_cols = data_columns[data_columns.column_type == 'num'].column_name.values

print(len(data_full))

262434


Сначала выделим тестовую и тренировочную части.

In [3]:
data_train = data_full[data_full.sample_ == 0]
data_test = data_full[data_full.sample_ == 1].drop(['price', 'sample_'], axis=1).values

Делим данные на еще один тест и трейн, для валидации,
чтобы проверить, как хорошо модель работает, до отправки submission на kaggle.

In [4]:
X = data_train.drop(['price', 'sample_'], axis=1).values
Y = data_train['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

def learn_model(model):        
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)    
    print (f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
    
def learn_model_log(model):
    model.fit(X_train,np.log(y_train+1))
    y_pred = np.exp(model.predict(X_test))
    print (f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

In [6]:
RANDOM_SEED = 42

Далее попробуем построить разные модели и посмотрим их метрики. Все модели закомментируем, чтобы не грузить ноутбук. В комментариях пропишем метрики. Оставим для загрузки только лучшую модель

# Наивная модель

In [7]:
#learn_model(LinearRegression())
#Точность модели по метрике MAPE: 87.83%

# Простые модели

In [8]:
# Построим линейную регрессию с логарифмированием целевой переменной
"""
lr = LinearRegression().fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(lr.predict(X_test))))*100:0.2f}%")
VERSION = 1
predict_test = np.exp(lr.predict(X_test))
predict_submission = np.exp(lr.predict(data_test))
"""
# Точность модели по метрике MAPE: 20.95%

'\nlr = LinearRegression().fit(X_train, np.log(y_train+1))\nprint(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(lr.predict(X_test))))*100:0.2f}%")\nVERSION = 1\npredict_test = np.exp(lr.predict(X_test))\npredict_submission = np.exp(lr.predict(data_test))\n'

In [9]:
# Random forest
"""
rf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1).fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf.predict(X_test))))*100:0.2f}%")
VERSION = 2
predict_test = np.exp(rf.predict(X_test))
predict_submission = np.exp(rf.predict(data_test))
"""
# Точность модели по метрике MAPE: 13.95%

'\nrf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1).fit(X_train, np.log(y_train+1))\nprint(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf.predict(X_test))))*100:0.2f}%")\nVERSION = 2\npredict_test = np.exp(rf.predict(X_test))\npredict_submission = np.exp(rf.predict(data_test))\n'

In [10]:
# Подбор параметров для Random forest
"""
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rf = RandomForestRegressor(random_state=RANDOM_SEED)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, 
                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
rf_random.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf_random.predict(X_test))))*100:0.2f}%")
VERSION = 8
predict_test = np.exp(rf.predict(X_test))
predict_submission = np.exp(rf.predict(data_test))
"""

'\nrandom_grid = {\'n_estimators\': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],\n               \'max_features\': [\'auto\', \'sqrt\'],\n               \'max_depth\': [int(x) for x in np.linspace(10, 110, num = 11)],\n               \'min_samples_split\': [2, 5, 10],\n               \'min_samples_leaf\': [1, 2, 4],\n               \'bootstrap\': [True, False]}\n\nrf = RandomForestRegressor(random_state=RANDOM_SEED)\nrf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, \n                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)\nrf_random.fit(X_train, np.log(y_train+1))\nprint(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf_random.predict(X_test))))*100:0.2f}%")\nVERSION = 8\npredict_test = np.exp(rf.predict(X_test))\npredict_submission = np.exp(rf.predict(data_test))\n'

In [11]:
#еще парочка экспериментов

#learn_model_log(KNeighborsRegressor(algorithm = 'ball_tree', weights = 'distance', p=1))
#Точность модели по метрике MAPE: 66.25% по-умолчанию
#Точность модели по метрике MAPE: 59.54% p=1
#Точность модели по метрике MAPE: 66.24% algorithm = 'ball_tree'
#Точность модели по метрике MAPE: 63.98% weights = 'distance'
#Точность модели по метрике MAPE: 66.25% leaf_size = 5
#Точность модели по метрике MAPE: 57.66% algorithm = 'ball_tree', weights = 'distance', p=1

#learn_model_log(KNeighborsRegressor(algorithm = 'ball_tree', weights = 'distance', p=1))
#Точность модели по метрике MAPE: 50.23%

#learn_model(DecisionTreeRegressor(random_state = 42))
#Точность модели по метрике MAPE: 110.55% max_depth=3, max_features=20
#Точность модели по метрике MAPE: 45.65% max_depth=15, max_features=5
#Точность модели по метрике MAPE: 22.29% max_depth=10
#Точность модели по метрике MAPE: 19.79% по-умолчанию
#Точность модели по метрике MAPE: 17.50% max_depth=15

#learn_model_log(DecisionTreeRegressor(max_depth=15, random_state = 42))
#Точность модели по метрике MAPE: 19.46% по-умолчанию
#Точность модели по метрике MAPE: 60.92% max_depth=3, max_features=20
#Точность модели по метрике MAPE: 38.89% max_depth=15, max_features=5
#Точность модели по метрике MAPE: 19.26% max_depth=10
#Точность модели по метрике MAPE: 16.56% max_depth=15

# Бустинг

In [12]:
# CatBoostRegressor
"""
cb = CatBoostRegressor(iterations = 5000, random_seed = RANDOM_SEED, eval_metric='MAPE', \
                            custom_metric=['R2', 'MAE'], silent=True,)
cb.fit(X_train, np.log(y_train+1), eval_set=(X_test, np.log(y_test+1)), verbose_eval=0, use_best_model=True)
cb.save_model('catboost_single_model_2_baseline.model')
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(cb.predict(X_test))))*100:0.2f}%")
VERSION = 3
predict_test = np.exp(cb.predict(X_test))
predict_submission = np.exp(cb.predict(data_test))
"""
# Точность модели по метрике MAPE: 13.51%
# Kaggle 19.66459

'\ncb = CatBoostRegressor(iterations = 5000, random_seed = RANDOM_SEED, eval_metric=\'MAPE\',                             custom_metric=[\'R2\', \'MAE\'], silent=True,)\ncb.fit(X_train, np.log(y_train+1), eval_set=(X_test, np.log(y_test+1)), verbose_eval=0, use_best_model=True)\ncb.save_model(\'catboost_single_model_2_baseline.model\')\nprint(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(cb.predict(X_test))))*100:0.2f}%")\nVERSION = 3\npredict_test = np.exp(cb.predict(X_test))\npredict_submission = np.exp(cb.predict(data_test))\n'

In [13]:
# GradientBoostingRegressor
"""
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(gb.predict(X_test))))*100:0.2f}%")
VERSION = 4
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(data_test))"""
# Точность модели по метрике MAPE: 13.68%
# Kaggle 18.86542

'\ngb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)\ngb.fit(X_train, np.log(y_train+1))\nprint(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(gb.predict(X_test))))*100:0.2f}%")\nVERSION = 4\npredict_test = np.exp(gb.predict(X_test))\npredict_submission = np.exp(gb.predict(data_test))'

In [14]:
# xgboost
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")
VERSION = 5
predict_test = np.exp(xb.predict(X_test))
predict_submission = np.exp(xb.predict(data_test))
# Точность модели по метрике MAPE: 13.31%
# Kaggle 18.84844

Точность модели по метрике MAPE: 13.49%


# Стекинг

In [15]:
"""
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
data_test = scaler.transform(data_test)

y_train = y_train 
y_test = y_test

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

def compute_meta_feature(regr, X_train, X_test, y_train, cv, data_test):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)

    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)

    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(data_test)

    return X_meta_train, X_meta_test, X_meta_pred

def generate_meta_features(regr, X_train, X_test, y_train, cv, data_test):
    features = [compute_meta_feature(regr, X_train, X_test, y_train, cv, data_test) for regr in tqdm(regr)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred

regr = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                             max_features=3, max_depth=19, bootstrap=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features([
                            regr,
                            RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1, max_depth=5, n_estimators=200),
                            ExtraTreesRegressor(random_state=RANDOM_SEED), 
                            RandomForestRegressor(random_state=RANDOM_SEED, max_depth=15) \
], X_train, X_test, y_train, cv, data_test)

def compute_metric(regr, X_train, y_train, X_test, y_test): 
    regr.fit(X_train, y_train) 
    y_test_pred = regr.predict(X_test) 
    return np.round(mape(y_test, y_test_pred)*100, 4)

print(f"Точность модели по метрике MAPE: {compute_metric(regr, stacked_features_train, y_train, stacked_features_test, y_test)}%")
VERSION = 6
predict_test = regr.predict(stacked_features_test)
predict_submission = regr.predict(stacked_features_pred)
"""
# Точность модели по метрике MAPE: 15.2942%
# Kaggle 22.59028

'\nscaler = StandardScaler() \nX_train = scaler.fit_transform(X_train) \nX_test = scaler.transform(X_test) \ndata_test = scaler.transform(data_test)\n\ny_train = y_train \ny_test = y_test\n\ncv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)\n\ndef compute_meta_feature(regr, X_train, X_test, y_train, cv, data_test):\n    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    \n\n    splits = cv.split(X_train)\n    for train_fold_index, predict_fold_index in splits:\n        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]\n        y_fold_train = y_train[train_fold_index]\n\n        folded_regr = clone(regr)\n        folded_regr.fit(X_fold_train, y_fold_train)\n\n        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)\n\n    meta_regr = clone(regr)\n    meta_regr.fit(X_train, y_train)\n\n    X_meta_test = meta_regr.predict(X_test)\n    X_meta_pred = meta_regr.predict(data_test)\n\n    return X_meta_train, X_meta

In [16]:
"""
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
data_test = scaler.transform(data_test)

y_train = y_train
y_test = y_test

cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

def compute_meta_feature(regr, X_train, X_test, y_train, cv, data_test):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)

    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)

    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(data_test)

    return X_meta_train, X_meta_test, X_meta_pred

def generate_meta_features(regr, X_train, X_test, y_train, cv, data_test):
    features = [compute_meta_feature(regr, X_train, X_test, y_train, cv, data_test) for regr in tqdm(regr)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred

regr = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                             max_features=3, max_depth=25, bootstrap=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features([
                            regr,
                            RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1, max_depth=5, n_estimators=200),
                            ExtraTreesRegressor(random_state=RANDOM_SEED),
                            GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300),
                            RandomForestRegressor(random_state=RANDOM_SEED, n_jobs = -1, max_depth=15) \
], X_train, X_test, y_train, cv, data_test)

def compute_metric(regr, X_train, y_train, X_test, y_test): 
    regr.fit(X_train, np.log(y_train+1))
    y_test_pred = np.exp(regr.predict(X_test))
    return np.round(mape(y_test, y_test_pred)*100, 4)

print(f"Точность модели по метрике MAPE: {compute_metric(regr, stacked_features_train, y_train, stacked_features_test, y_test)}%")
VERSION = 7
predict_test = np.exp(regr.predict(stacked_features_test))
predict_submission = np.exp(regr.predict(stacked_features_pred))
"""
# Точность модели по метрике MAPE: 13.8579%
# Kaggle 20.63917

'\nscaler = StandardScaler() \nX_train = scaler.fit_transform(X_train) \nX_test = scaler.transform(X_test) \ndata_test = scaler.transform(data_test)\n\ny_train = y_train\ny_test = y_test\n\ncv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)\n\ndef compute_meta_feature(regr, X_train, X_test, y_train, cv, data_test):\n    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    \n\n    splits = cv.split(X_train)\n    for train_fold_index, predict_fold_index in splits:\n        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]\n        y_fold_train = y_train[train_fold_index]\n\n        folded_regr = clone(regr)\n        folded_regr.fit(X_fold_train, y_fold_train)\n\n        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)\n\n    meta_regr = clone(regr)\n    meta_regr.fit(X_train, y_train)\n\n    X_meta_test = meta_regr.predict(X_test)\n    X_meta_pred = meta_regr.predict(data_test)\n\n    return X_meta_train, X_meta

## Выводы следующие:
1. Catboost достаточно хороший алгоритм, и работает даже с пропусками.
2. Превзойти результат Catboost удалось только с xgboost. Его и будем использовать в соревовании
3. Метрику сильно улучшает логарифмирование целевой переменной
4. Обучение больших датасетов (250к обьектов и 20+ признаков) требует хороших ресурсов. Не все можно запустить на кагле (например, глубина дерева, поиск гиперпараметров и подобное вылетают с ошибкой нехватки памяти...)

## Что бы мы сделали еще, но из-за ограниченности по времени, оставим в таком виде:
1. Изменить способ кодирования признаков
2. Попробовать использовать признак с моделями авто, который мы удалили
3. Прологарифмировать все числовые признаки
4. Сделать целевую переменную категориальной (в диапазоне +-2сигма с шагом 10к) и применить модели классификации
5. Попробовать стекинг лучших алгоритмов GradientBoostingRegressor, XGBRegressor и Catboost

# Submission

In [17]:
#predict_test = np.exp(model.predict(X_test))
#predict_submission = np.exp(model.predict(data_test))

data_sample['price'] = predict_submission
data_sample.to_csv(f'submission_2_v{VERSION}.csv', index=False)
data_sample.head(10)

,sell_id,price
0,1100575026,6.647342e+05
1,1100549428,9.433365e+05
2,1100658222,9.253898e+05
3,1100937408,7.476332e+05
4,1101037972,8.538589e+05
5,1100912634,8.225001e+05
6,1101228730,7.155538e+05
7,1100165896,4.709632e+05
8,1100768262,1.925684e+06
9,1101218501,1.045331e+06
